In [26]:
import tensorflow as tf

In [4]:
from tensorflow.keras.models import load_model
import pickle
import pandas as pd,numpy as np

In [28]:
#Load the trained model
model = load_model('model.h5')

#load the encoder and scaler
with open('onehot_encoder_geo.pkl','rb') as file:
    en_geo = pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    en_gender = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [29]:
input_data = {
    'CreditScore':600,	
    'Geography':'France',
    'Gender':'Male',
 	'Age':40,
    'Tenure':3,
	'Balance':60000,
 	'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50
}

In [30]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50


In [31]:
geo_en = en_geo.transform([[ input_data['Geography'] ]])
geo_en_df = pd.DataFrame(geo_en.toarray(),columns = en_geo.get_feature_names_out())
geo_en_df

c:\Users\user\OneDrive\Desktop\Projects Deep Learning\Churn_ANN\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [32]:
input_df['Gender'] = en_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50


In [33]:
input_df = pd.concat([input_df.drop(['Geography'],axis = 1),geo_en_df],axis = 1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50,1.0,0.0,0.0


In [35]:
input_df.shape

(1, 12)

In [36]:
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -1.74529639,  1.00150113,
        -0.57946723, -0.57638802]])

In [37]:
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step


array([[0.0614697]], dtype=float32)

In [39]:
pred_prob = prediction[0][0]
pred_prob 

np.float32(0.061469704)

In [40]:
if pred_prob >0.5:
    print('churn')
else:
    print('Not churn')

Not churn
